#  Fine-tuning vs RAG 비교

##  파인튜닝(Fine-tuning)

- **필요한 데이터**: 질문-답변 쌍 (Q&A) 형식의 `.jsonl` 파일. 허깅페이스와 GPT 모두 학습시킬 때 jsonl 파일형태를 사용합니다.
- **작동 방식**: 모델에게 특정 유형의 질문과 그에 대한 정답을 "학습"
- **활용 사례**: 정형화된 응답, 고정된 문체 또는 스타일 학습에 적합
- **단점**:
  - 질문/답변 쌍을 미리 만들어야 함
  - 데이터 변경 시마다 재학습 필요
  - 학습에 비용과 시간이 듦

---

##  RAG (Retrieval-Augmented Generation)

- **필요한 데이터**: 일반 문서(PDF, txt 등)
- **작동 방식**: 질문을 하면 관련 내용을 문서에서 찾아 LLM이 답변 생성
- **활용 사례**: 문서 기반 질의응답, 문서 요약, 실시간 검색 기반 답변
- **장점**:
  - 질문/답변 쌍 없이 사용 가능
  - 문서만 준비하면 곧바로 작동
  - 문서 추가/수정이 쉬움 (재학습 불필요)

---

##  요약 비교

| 항목 | Fine-tuning | RAG |
|------|-------------|-----|
| 학습 데이터 | 질문 + 답변 | 문서만 |
| 학습 필요 | 필요함 | 필요 없음 |
| 실시간 업데이트 | 어렵다 | 가능하다 |
| 유연한 질문 처리 | 제한적 | 유연함 |
| 구축 난이도 | 높음 | 낮음 (LangChain 등으로 구현 쉬움) |

---

##  결론

> **질문-답변 쌍이 없다면 Fine-tuning보다는 RAG를 추천.**

# 랭체인 RAG

In [ ]:
# 설치 
%pip install langchain_community # langchain 커뮤니티 확장
%pip install pypdf # PDF 읽기
%pip install faiss-cpu # 임베딩

In [11]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, JSONLoader, CSVLoader, WebBaseLoader, ImageCaptionLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter # RecursiveCharacterTextSplitter 는 문장이 잘리면 안되는 경우에 사용(문장이 예쁘게 잘리게 도와줌)
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS # vector DB
from langchain.chains import RetrievalQA # RAG. 질문-답변(질문을 넣으면 관련된 문서를 찾아서 답변 => 유사도 검색)
import warnings 
warnings.simplefilter('ignore')

### 문서읽기

In [18]:
pdfloader = PyPDFLoader('../download/예시사업보고서.pdf')
document = pdfloader.load()
len( document)

3

In [19]:
for doc in document[2:3]:
    print( doc.page_content)
    print('----------------')

Sample Company 
Balance Sheet 
September 30, 2021 
 
Assets  Liabilities and Stockholders’ Equity 
Current Assets:  Current Liabilities:  
Cash $ 1,550 Accounts payable $60 
Accounts receivable 770 Interest payable   80 
Supplies 40 Wages payable   100 
Total current assets 2,360 Income taxes payable 
Utilities payable 
  405 
  250 
  Total Current Liabilities   895 
     
Equipment  12,000 Long-term Notes Payable   8,000 
Less: Accumulated deprec. (1,300)   
 10,700 Owners' Equity:  
  Owners’ capital   2,900 
  Retained earnings   1,265 
  Total equity   
  4,165  
 
Total assets 
 
$13,060 
Total liabilities and 
owners' equity 
 
  $13,060
----------------


### 문서 자르기

In [20]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20) # overlap: 겹치는 부분(문장이 잘리는 것을 방지하기 위해서 최대 20자 겹침)
splittedDoc = splitter.split_documents( document) # 문서 나누기

In [21]:
len(splittedDoc)

6

### Embedding

In [22]:
embedding = OpenAIEmbeddings( model='text-embedding-3-small')
## 임베딩 모델은 
# 고품질(정확도 우선): text-embedding-3-large
# 비용/속도 우선(트레이드오프 있음): text-embedding-3-small
# 기존 코드에 있던 text-embedding-ada-002도 동작하지만 최신 권장 모델로 교체하는 게 낫습니다.
## 벡터db에 저장되는 종류 # 벡터 디비에 저장할 때는 아래와 같은 3가지 정보를 같이 저장 # 나중에 유사도 검색 시 몇 페이지에 어디에 있는 내용이었는지를 다 알 수 있음
## 1. page_content : embdding 저장
## 2. 원본텍스트도 저장
## 3. 각종정보:pdf이름 페이지번호,..
vdb = FAISS.from_documents( splittedDoc, embedding)

In [ ]:
# vdb.save_local('myfaiss') # 메모리에 저장된 벡터db를 로컬에 저장
# FAISS.load_local('myfaiss') # 로컬에 저장된 벡터db를 메모리에 로드

In [23]:
gpt = ChatOpenAI( model='gpt-5-nano', temperature=0)
retriever = vdb.as_retriever() # vdb.as_retriever(k=4) 기본 값은 k=4 로 유사도 검색을 진행합니다
qa = RetrievalQA.from_chain_type( llm=gpt, chain_type='stuff',
                        retriever=retriever) # model 과 retriever 연결
query = "제공된 문서의 내용을 요약해줘"
result = qa.invoke( {'query':query})
print( result )

{'query': '제공된 문서의 내용을 요약해줘', 'result': "다음은 제공된 Sample Company의 재무제표 내용을 간략히 요약한 것입니다 (2021년 9월 30일 종료).\n\n- 구성 문서\n  - 손익계산서(제품형)\n  - 손익계산서(서비스형)\n  - 포괄손익계산서? 아닌, 이익잉여금표? (Statement of Retained Earnings)\n  - 대차대조표(Balance Sheet)\n\n- 손익계산서 요약\n  - 제품형:\n    - 매출원가( COGS ): 4,125\n    - 매출액( Sales revenue ): 6,875\n    - 매출총이익: 2,750\n    - 영업비용 합계: 1,360\n    - 영업이익: 1,390\n    - 이자비용: 40\n    - 세전이익: 1,350\n    - 법인세비용: 405\n    - 순이익: 945\n  - 서비스형:\n    - 서비스매출: 2,750\n    - 기타 비용은 동일하게 영업비용 1,360, 영업이익 1,390\n    - 이자비용: 40\n    - 세전이익: 1,350\n    - 법인세비용: 405\n    - 순이익: 945\n  - 관찰: 매출 규모가 다르지만 두 형식 모두 순이익 945로 동일하게 나옴. 비용구조는 동일.\n\n- 이익잉여금(Statement of Retained Earnings) 요약\n  - 기초 Retained Earnings: 820\n  - 순이익: 945\n  - 배당금: (500)\n  - 기말 Retained Earnings: 1,265\n  - 해석: 순이익에서 배당금을 차감하고 누적 증가.\n\n- 대차대조표 요약\n  - 자산\n    - 현재자산 합계: 2,360\n      - 현금 1,550; 매출채권 770; 재고/소모자산(소모품) 40\n    - 비유동자산: 설비 12,000, 감가누계 1,300\n      - 순설비: 10,700\n    - 총자산: 13,060\n  - 부채 및 자본\